In [1]:
import pandas as pd
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

C:\Users\owner\anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# Read csv file into dataframe
df = pd.read_csv(
    Path('Resources/PS_20174392719_1491204439457_log.csv'))

df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0


In [3]:
df.shape

(6362620, 11)

In [4]:
sample_dataframe = df.sample(n=10000)
sample_dataframe.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
4703243,331,PAYMENT,8283.81,C74304366,45698.00,37414.19,M36159356,0.00,0.00,0,0
643239,35,PAYMENT,24999.20,C874383203,0.00,0.00,M1020907807,0.00,0.00,0,0
3869882,283,PAYMENT,15694.07,C1639938567,8086.00,0.00,M1219609312,0.00,0.00,0,0
4942511,350,PAYMENT,24055.22,C2121696514,160111.89,136056.67,M1207484963,0.00,0.00,0,0
6064339,505,PAYMENT,8626.06,C239567860,10490.00,1863.94,M901356354,0.00,0.00,0,0


In [5]:
sample_dataframe.drop('nameOrig', axis=1, inplace=True)
sample_dataframe.drop('nameDest', axis=1, inplace=True)
sample_dataframe.drop('isFlaggedFraud', axis=1, inplace=True)



In [6]:
sample_dataframe.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
4703243,331,PAYMENT,8283.81,45698.00,37414.19,0.00,0.00,0
643239,35,PAYMENT,24999.20,0.00,0.00,0.00,0.00,0
3869882,283,PAYMENT,15694.07,8086.00,0.00,0.00,0.00,0
4942511,350,PAYMENT,24055.22,160111.89,136056.67,0.00,0.00,0
6064339,505,PAYMENT,8626.06,10490.00,1863.94,0.00,0.00,0


In [7]:
sample_dataframe.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
count,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00
mean,243.97,180982.06,853276.05,875056.56,1085752.07,1223453.13,0.00
std,141.85,568585.09,2984563.75,3018913.24,3557426.52,4132351.08,0.02
min,1.00,8.26,0.00,0.00,0.00,0.00,0.00
25%,156.00,13368.76,0.00,0.00,0.00,0.00,0.00
50%,239.00,75934.13,13803.50,0.00,131830.38,217905.70,0.00
75%,335.00,207255.89,104717.85,144226.28,952330.89,1121106.73,0.00
max,718.00,22730641.11,32018440.44,32554680.53,192541564.92,249370946.22,1.00


In [8]:
sample_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 4703243 to 115206
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            10000 non-null  int64  
 1   type            10000 non-null  object 
 2   amount          10000 non-null  float64
 3   oldbalanceOrg   10000 non-null  float64
 4   newbalanceOrig  10000 non-null  float64
 5   oldbalanceDest  10000 non-null  float64
 6   newbalanceDest  10000 non-null  float64
 7   isFraud         10000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 703.1+ KB


In [9]:
sample_dataframe = pd.get_dummies(sample_dataframe)

sample_dataframe

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
4703243,331,8283.81,45698.00,37414.19,0.00,0.00,0,0,0,0,1,0
643239,35,24999.20,0.00,0.00,0.00,0.00,0,0,0,0,1,0
3869882,283,15694.07,8086.00,0.00,0.00,0.00,0,0,0,0,1,0
4942511,350,24055.22,160111.89,136056.67,0.00,0.00,0,0,0,0,1,0
6064339,505,8626.06,10490.00,1863.94,0.00,0.00,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1738992,161,127010.48,916.00,127926.48,1587428.96,1460418.49,0,1,0,0,0,0
1122079,130,106468.69,2682084.93,2788553.62,1835035.77,1728567.07,0,1,0,0,0,0
5847910,402,225.81,29340.07,29114.26,0.00,0.00,0,0,0,0,1,0
272290,15,413259.02,9991.00,0.00,0.00,413259.02,0,0,0,0,0,1


In [10]:
X = sample_dataframe.drop(columns=['isFraud'])
y = sample_dataframe['isFraud']

print(sample_dataframe.isFraud.value_counts())

0    9995
1       5
Name: isFraud, dtype: int64


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train


,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
5425331,378,180071.86,0.00,180071.86,3495620.85,3315548.99,1,0,0,0,0
4246397,306,116722.39,4200641.68,4317364.07,1548641.12,1431918.73,1,0,0,0,0
4908221,349,102354.13,1827.00,104181.13,6646167.24,6543813.11,1,0,0,0,0
3779807,280,15405.29,51908.00,36502.71,0.00,0.00,0,0,0,1,0
5422226,378,2391.73,0.00,0.00,0.00,0.00,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
206266,13,100376.52,11019.00,0.00,55034.93,155411.45,0,1,0,0,0
2463956,203,83416.66,11157226.13,11240642.78,580959.48,497542.82,1,0,0,0,0
4269646,307,20204.00,11394.00,0.00,0.00,0.00,0,0,0,1,0
379789,17,349171.23,64223.00,0.00,373600.57,722771.80,0,1,0,0,0


In [13]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [14]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
random_oversampler = RandomOverSampler(random_state=1)

In [16]:
X_resampled, y_resampled = random_oversampler.fit_resample(X_train_scaled, y_train)

In [17]:
pd.Series(y_resampled).value_counts()

1    6998
0    6998
Name: isFraud, dtype: int64

In [18]:
clf = LazyClassifier(verbose=0, custom_metric=None)
models,predictions = clf.fit(X_resampled, X_test_scaled, y_resampled, y_test)

print(models)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [01:06<00:00,  2.31s/it]


                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
LinearDiscriminantAnalysis         0.89               0.78     0.78      0.94   
RidgeClassifier                    0.89               0.78     0.78      0.94   
RidgeClassifierCV                  0.89               0.78     0.78      0.94   
NuSVC                              0.84               0.75     0.75      0.91   
NearestCentroid                    0.82               0.74     0.74      0.90   
GaussianNB                         0.68               0.68     0.68      0.81   
Perceptron                         1.00               0.67     0.67      1.00   
AdaBoostClassifier                 1.00               0.67     0.67      1.00   
XGBClassifier                      1.00               0.67     0.67      1.00   
DecisionTreeClassifier             1.00               0.67     0.67      1.00   
RandomForestClassifier      